In [1]:
# 1) Información de TensorFlow y GPU disponibles
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("¿Compilado con soporte CUDA?:", tf.test.is_built_with_cuda())
print("GPUs visibles por TensorFlow:", tf.config.list_physical_devices('GPU'))

# 2) Detalles de la versión de CUDA y cuDNN usadas por TensorFlow
build_info = tf.sysconfig.get_build_info()
print("Versión CUDA en TF build:", build_info.get('cuda_version', 'desconocida'))
print("Versión cuDNN en TF build:", build_info.get('cudnn_version', 'desconocida'))

# 3) Prueba rápida: crea un tensor y ponlo en la GPU (si hay)
if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        a = tf.random.uniform((1000, 1000))
        b = tf.matmul(a, a)
    print("Operación matmul en GPU completada, tensor shape:", b.shape)
else:
    print("No se detectó GPU, la operación se hará en CPU.")




TensorFlow version: 2.10.0
¿Compilado con soporte CUDA?: True
GPUs visibles por TensorFlow: []
Versión CUDA en TF build: 64_112
Versión cuDNN en TF build: 64_8
No se detectó GPU, la operación se hará en CPU.


In [4]:
import struct
print(struct.calcsize("P") * 8, "bits")

64 bits
